In [10]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2"


from torchvision.utils import make_grid, save_image
import torchvision.models as models
from handler import geozoom_handler
import torch.nn.functional as F
import matplotlib.pyplot as plt
from copy import deepcopy
import numpy as np
import random
import torch
import json
import PIL


from sa import *
from fc_net import *
from helpers import *
from utils2 import *

In [11]:
# random.sample(range(0, len(image_names)), 5)

In [12]:
# image_names = get_png_names("../pooling/data/MEX2/")#[100:105]
image_names = get_png_names("../pooling/data/MEX2/")
image_indices = random.sample(range(0, len(image_names)), 300)
image_names = [image_names[i] for i in range(len(image_names)) if i in image_indices]
y = get_migrants("../pooling/data/migration_data.json" , image_names)

300 municipalities.


In [4]:
# for i in image_names:
#     print(load_inputs(i).shape)

In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet18 = models.resnet18().to(device)
attn_model = attnNet(resnet = resnet18).to(device)
lr = .001
criterion = torch.nn.L1Loss(reduction = 'mean')
attn_optimizer = torch.optim.Adam(attn_model.parameters(), lr = lr)

fc_model = fc_net(resnet = resnet18).to(device)
fc_optimizer = torch.optim.Adam(fc_model.parameters(), lr = .01)

butler = geozoom_handler(attn_model, 
                         fc_model, device, 
                         criterion, 
                         attn_optimizer, 
                         fc_optimizer, 
                         num_thresholds = 10,
                         reduction_percent = .90,
                         convergence_dims = (358, 284),
                         plot = False, 
                         v = False)

In [17]:
attn_model.sa

SelfAttention(
  (query_conv): Conv2d(256, 32, kernel_size=(1, 1), stride=(1, 1))
  (key_conv): Conv2d(256, 32, kernel_size=(1, 1), stride=(1, 1))
  (value_conv): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
  (softmax): Softmax(dim=-1)
)

In [19]:
BATCH_SIZE = 1
SPLIT = .60

train_dl, val_dl = butler.prep_attn_data(image_names, y, SPLIT, BATCH_SIZE)

In [21]:
# for i,o in val_dl:
#     print(load_inputs(i[0]).shape, o)

In [ ]:
butler.train_attn_model(train_dl, val_dl)

Epoch:  0
  Training Loss:  1425.1291727807786
  Validation Loss:  1737.4661228815714
Loss thresholds for training:  [0.0, 173.74661228815714, 347.4932245763143, 521.2398368644714, 694.9864491526286, 868.7330614407857, 1042.4796737289428, 1216.2262860171, 1389.972898305257, 1563.7195105934143]
Starting from threshold:  9  with value:  1563.7195105934143


Epoch:  1
  Training Loss:  1367.489933649699
  Validation Loss:  1693.103741836548


Epoch:  2
  Training Loss:  1344.431157599555
  Validation Loss:  1669.5068967183431


Epoch:  3
  Training Loss:  1335.6826685587564
  Validation Loss:  1664.6083841959635


Epoch:  4
  Training Loss:  1329.4608574761285
  Validation Loss:  1655.1432375590007


Epoch:  5
  Training Loss:  1322.4259048461913
  Validation Loss:  1643.8307373046875


Epoch:  6
  Training Loss:  1311.729699198405
  Validation Loss:  1631.387195332845


Epoch:  7
  Training Loss:  1307.433081902398
  Validation Loss:  1631.795947011312


Epoch:  8
  Training Loss:  1304.

/opt/conda/envs/rapids/lib/python3.7/site-packages/torch/nn/functional.py:3328: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")


Reinitializing random weights.


Epoch:  42
  Training Loss:  1217.6345035129123
  Validation Loss:  1571.5354062398276


Epoch:  43
  Training Loss:  1202.7359585232205
  Validation Loss:  1572.0446230570476


Epoch:  44
  Training Loss:  1189.9647952609591
  Validation Loss:  1573.3786153157553


Epoch:  45
  Training Loss:  1170.2993370903862
  Validation Loss:  1569.170051320394


Epoch:  46
  Training Loss:  1155.550681898329
  Validation Loss:  1581.2590558369955


Epoch:  47
  Training Loss:  1143.3155339558919
  Validation Loss:  1564.398506673177


Epoch:  48
  Training Loss:  1158.104810587565
  Validation Loss:  1561.1119468688964


Epoch:  49
  Training Loss:  1117.9909598456488
  Validation Loss:  1570.2025967915854




In [ ]:
def val_outside(cur_image, output):

    model = butler.attn_model
    keys = list(butler.threshold_weights.keys())
    
    print("Keys: ", keys)

    # If there are acutally weights in the threshold dictionary (i.e. if the training data has been clipped at least once)...
    if len(keys) > 0:

        # For the weights equivalent to each clip...
        for k in butler.threshold_weights.keys():

            # Load the weights that coorespond to that clip
            model.load_state_dict(butler.threshold_weights[k])

            # and set phasers to stun...
            model.eval()

            # If it is the last key in the dictionary...
            if k == keys[-1]:
                
                print("Last key")
                
                print("Shape before clip: ", cur_image.shape)

                # Do all of the things to get the attention map and clip the image
                IM_SIZE = (cur_image.shape[2], cur_image.shape[3])
                gradcam, attn_heatmap = get_gradcam(model, IM_SIZE, cur_image.cuda(), target_layer = butler.model.sa) 
                cur_image, new_dims = butler.clip_input(cur_image, attn_heatmap)   
                
                print("Shape after clip: ", cur_image.shape)

                # Now that you have the image according to all of the past clips, run it through the current state of the model (mid-threshold weights)
                butler.model.eval()
                y_pred = butler.model(cur_image.cuda())
                print("PRED: ", y_pred.item())
                loss = butler.criterion(y_pred, output.view(-1,1).to(butler.device))
                print("LOSS: ", loss.item())
#                 butler.running_val_loss += l oss.item() 

            # If it's not the last key in the dictionary, clip it to the k weights and send it back through again
            else:

                IM_SIZE = (cur_image.shape[2], cur_image.shape[3])
                gradcam, attn_heatmap = get_gradcam(model, IM_SIZE, cur_image.cuda(), target_layer = butler.model.sa) 
                cur_image, new_dims = butler.clip_input(cur_image, attn_heatmap)    

    # If there are no threshold weights, just run it through the model as is
    else:

        butler.model.eval()
        y_pred = butler.model(cur_image.cuda())
        print("PRED: ", y_pred.item())
        loss = butler.criterion(y_pred, output.view(-1,1).to(butler.device))
        print("LOSS: ", loss.item())
#         butler.running_val_loss += loss.item() 

In [29]:
for i, o in val_dl:
    inp = butler.prep_input(i)
    val_outside(inp, o)

Keys:  [9]
Last key
Shape before clip:  torch.Size([1, 3, 1325, 1658])
Shape after clip:  torch.Size([1, 3, 1192, 1492])
PRED:  310.58905029296875
LOSS:  915.4109497070312
Keys:  [9]
Last key
Shape before clip:  torch.Size([1, 3, 539, 579])
Shape after clip:  torch.Size([1, 3, 485, 521])
PRED:  338.44354248046875
LOSS:  86.44354248046875
Keys:  [9]
Last key
Shape before clip:  torch.Size([1, 3, 462, 699])
Shape after clip:  torch.Size([1, 3, 415, 629])
PRED:  286.9433288574219
LOSS:  259.0566711425781
Keys:  [9]
Last key
Shape before clip:  torch.Size([1, 3, 355, 308])
Shape after clip:  torch.Size([1, 3, 224, 224])
PRED:  369.9192199707031
LOSS:  59.919219970703125
Keys:  [9]
Last key
Shape before clip:  torch.Size([1, 3, 1909, 1751])
Shape after clip:  torch.Size([1, 3, 1718, 1575])
PRED:  276.0505676269531
LOSS:  549.949462890625
Keys:  [9]
Last key
Shape before clip:  torch.Size([1, 3, 144, 210])
Shape after clip:  torch.Size([1, 3, 144, 14])
PRED:  272.150390625
LOSS:  220.1503906

In [20]:
butler.threshold_weights.keys()

dict_keys([9])

In [ ]:
butler.image_sizes

In [ ]:
butler.threshold_weights.keys()

In [ ]:
for i,o in train_dl:
    print(load_inputs(i[0]).shape)

In [ ]:
641 * .70

In [ ]:
263 - 

In [8]:
for i, o in train_dl:
    
    muni_id = i[0].split("/")[4]
    
    plt.imshow(butler.prep_input(i).detach().cpu()[0].permute(1,2,0))
    plt.title(butler.image_sizes[muni_id])
    plt.savefig(f"test{muni_id}.png")
    
    plt.clf()

<Figure size 432x288 with 0 Axes>

In [9]:
# # for i in range(50):
# while handler.threshold_index > 0:

#     for impath, output in train_dl:

#         # Prep the input and pass it to the trainer (this could easily be done in one step eventually if ya want)
#         input = handler.prep_input(impath)
#         handler.train(input, output)
    
#     handler.end_epoch(train_dl, val_dl = None)

In [8]:
handler.image_sizes

{'484002003': (0, 224, 0, 224),
 '484016045': (0, 224, 0, 224),
 '484020489': (0, 224, 0, 224),
 '484020254': (0, 224, 0, 224)}

In [11]:
list(butler.threshold_weights.keys())

[6]

In [2]:
# for i, o in val_dl:
#     print(i, o)

In [1]:
# def run_validation(attn_model, fc_model, input, weights_dict, plot = False):
    
#     muni_id = input[0].split("/")[4]
#     cur_image = load_inputs(input[0])    
    
#     for k in weights_dict.keys():
                
#         if k != 'fc':
            
#             model = attn_model
#             model.load_state_dict(weights_dict[k])
#             model.eval()
#             IM_SIZE = (cur_image.shape[2], cur_image.shape[3])
#             gradcam, attn_heatmap = get_gradcam(model, IM_SIZE, cur_image.cuda()) 
#             cur_image, new_dims = butler.clip_input(cur_image, attn_heatmap)
            
#             if plot == True:
            
#                 plot_gradcam(gradcam)
#                 plt.savefig(f"threshold{k}_muni{muni_id}.png")
#                 plt.clf()
            
        
#         if k == 'fc':
            
#             model = fc_model
#             model.load_state_dict(weights_dict[k])
#             model.eval()
#             return model(cur_image.cuda()).item()
            

        
# for i, o in train_dl:

#     print(round(run_validation(attn_model, fc_model, i, butler.threshold_weights), 2), "     ", o.item())
    